# Análisis de sentimientos

__Qué necesitarás?__

* pandas : librería de manipulación y análisis de datos de código abierto rápida, potente, flexible y fácil de usar, construida sobre el lenguaje de programación Python
* numpy : paquete fundamental para la computación científica en Python. Es una biblioteca de Python que proporciona un objeto de matriz multidimensional, varios objetos derivados (como matrices y matrices enmascaradas) y una variedad de rutinas para operaciones rápidas en matrices, que incluyen manipulación matemática, lógica, clasificación, selección, transformadas discretas de Fourier, álgebra lineal básica, operaciones estadísticas básicas, simulación aleatoria ...
* train_test_split: función de sklearn.model_selection que permite particionar la data de entrenamiento y testeo para fines de entrenamiento en modelos de ML e IA
* test_size: patron para muestra de prueba. El dato se define entre 0 y 1
* random_state: semilla para generador de números aleatorios, lo que permite reporducir la función y garantizar la constancia del resultado obtenido
* CountVectorizer : función de la librería de sklearn.feature_extraction.text, la cual permite la extracción de características de texto ---> https://qu4nt.github.io/sklearn-doc-es/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
* token_pattern : Expresión regular que denota lo que constituye un «token», sólo se utiliza si analyzer ---> {“word”, “char”, “char_wb”} o callable, default=”word”
* lowercase : Convierte todos los caracteres en minúsculas antes de la tokenización
* PorterStemmer : función de nltk.stem.porter la cual permite eliminar los afijos morfológicos de las palabras, dejando solo la raíz de la palabra ----> https://www.nltk.org/howto/stem.html
* build_analyzer : Devuelve un invocable para procesar los datos de entrada. Los identificadores invocables que manejan el preprocesamiento, la tokenización y la generación de n-gramas (subsecuencia de n elementos de una secuencia dada).
* GridSearchCV : función de sklearn.model_selection conocida como instancia que permite realizar busqueda exhaustiva sobre valores de parámetros específicos para un estimador ---> https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html 
* estimator (GridSearchCV) : implementa la interfaz del estimador scikit-learn
* param_grid (GridSearchCV) : Diccionario con nombres de parámetros ( str) como claves y listas de configuraciones de parámetros para probar como valores, o una lista de dichos diccionarios, en cuyo caso se exploran las cuadrículas que abarca cada diccionario de la lista. Esto permite buscar en cualquier secuencia de ajustes de parámetros.
* cv (GridSearchCV) : Generador de validación cruzada. Determina la estrategia de división de validación cruzada. Algunas de las entradas posibles para cv son: Ninguno, para usar la validación cruzada de 5 veces predeterminada; entero, para especificar el número de pliegues.
* scoring (GridSearchCV) : Estrategia para evaluar el rendimiento del modelo de validación cruzada en el conjunto de prueba.
* refit (GridSearchCV) :  booleano que realiza o no ajuste de un estimador utilizando los mejores parámetros encontrados en todo el conjunto de datos
* return_train_score (GridSearchCV) : booleano que devuelve puntajes de entrenamiento
* Pipeline : función de sklearn.pipeline que actua como canalizador para ensamblar varios pasos que se pueden validar de forma cruzada mientras se configuran diferentes parámetros ----> https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html 
* BernoulliNB : función de sklearn.naive_bayes conocido como clasificador Naive Bayes para modelos multivariados de Bernoulli. BernoulliNB está diseñado para características binarias/booleanas ----> https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html 
* confusion_matrix : función de sklearn.metrics que permite evaluar la precisión de una clasificación.
* variable y: variable dependiente cuyos valores dependen de los que tomen otra(s) variable(s)
* Variable x: Variable independiente 

# Caso de uso

El archivo `data.txt` contiene una serie de comentarios sobre productos
de la tienda de amazon, los cuales están etiquetados como positivos (1), negativos (0)
o indterminados (NULL).

Haciendo uso de Naive Bayes, realice análisis de sentimientos

# Pasos a Seguir

1. Cargue los datos haciendo uso de la librería de pandas, indicando que la columna 0 corresponde al mensaje y la columna 1 corresponde a su etiqueta.
2. Obtenga los grupos de mensajes (etiquetados y sin etiquetar), definiendo como variable independependiente el mensaje y como dependiente la etiqueta
3. Preparar conjunto de datos: Partione la data, de tal manera que obtenga la data de entrenamiento para variable independiente y dependiente, a partir de los grupos de datos etiquetados.Es de resaltar que, la semilla del generador de números aleatorios debe ser 12345 y debe usar el 10% de patrones para la muestra de prueba
4. Cunstruya un analizador de palabras: crear _stemmer_, compilar un _vectorizer_, crear instancia a partir del _vectorizer_ haciendo uso de build_analyzer, ejecutar el analizador
5. Crear una instancia de CountVectorizer que use el analizador de palabras antes mencionado: Esta instancia debe retornar una matriz binaria. El límite superior para la frecuencia de palabras es del 100% y un límite inferior de 5 palabras. Solo deben analizarse palabras conformadas por letras.
6. Crear pipeline que contenga el CountVectorizer mencionado en el ítem anterior y el modelo de BernoulliNB, con el objetivo de definir el estimador de GridSearchCV previo al entrenamiento.
7. Definir un diccionario de parámetros para el GridSearchCV (param_grid). Se deben considerar 10 valores entre 0.1 y 1.0 para el parámetro alpha de BernoulliNB.
8. Definir una instancia de GridSearchCV con el pipeline y el diccionario de parámetros. Use cv=5, y "accuracy" como métrica de evaluación
9. Buscar la mejor combinación de regresores: entrenar instancia de GridSearchCV
10. Evaluar el modelo con los datos de entrenamiento y de prueba usando la matriz de confusión de sklearn.metrics
11. Pronosticar la polaridad del sentimiento para los datos


In [1]:
# Importamos librerías
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

In [29]:
# 1
def read_data(input, columns_data):
    df = pd.read_csv(
        input, 
        sep="\t", 
        header=None, 
        names=columns_data
    )
    return df

#2

def group_data_for_label(df):
    df_tagged = df[(df["etiqueta"]>=0)]
    df_untagged = df[(df["etiqueta"].isna())]

    x_tagged = df_tagged["mensaje"]
    y_tagged = df_tagged["etiqueta"]

    x_untagged = df_untagged["mensaje"]
    y_untagged = df_untagged["etiqueta"]

    return x_tagged, y_tagged, x_untagged, y_untagged

#3

def data_set_preparation(x_tagged, y_tagged):

    x_train, x_test, y_train, y_test = train_test_split( 
        x_tagged, 
        y_tagged,
        test_size=0.1, 
        random_state=12345
    )
    return x_train, x_test, y_train, y_test

#4
def building_word_analyzer():

    stemmer = PorterStemmer()
    #________________________________
    vectorizer= CountVectorizer(
        analyzer="word",
        token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b", 
        lowercase=True
    )
    #_________________________________
    analyzer =vectorizer.build_analyzer()

    return lambda x: (stemmer.stem(w) for w in analyzer(x))

# 5

def instance_countvectorizer(analyzer):
    countVectorizer = CountVectorizer(
        analyzer=analyzer,
        lowercase=True,
        stop_words="english",
        token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b", 
        binary=True, 
        max_df=1.0, 
        min_df=5
    )
    return countVectorizer

#6

def create_pipeline(countVectorizer):
    pipeline = Pipeline(
        steps=[
            ("countVectorizer", countVectorizer),
            ("bernoulli", BernoulliNB()),
        ],
    )
    return pipeline

#7 y 8

def create_instance_gridsearchCV(pipeline, x_train, y_train):
    #7__________________________________
    param_grid = {
        "bernoulli__alpha": np.arange(0.1,1.01, 0.1),
    }
    #8_________________________________
    gridSearchCV = GridSearchCV(
        estimator=pipeline,
        param_grid= param_grid,
        cv=5,
        scoring="accuracy",
        refit=True,
        return_train_score=True,
    )
    #9________________________________
    gridSearchCV.fit(x_train, y_train)

    return gridSearchCV

#10
def assess_model(gridSearchCV,x_train, x_test, y_train, y_test ):

    #trein____________________________________________
    cfm_train = confusion_matrix(
        y_true=y_train,
        y_pred=gridSearchCV.predict(x_train),
    )

    #test______________________________________________
    cfm_test = confusion_matrix(
        y_true=y_test,
        y_pred=gridSearchCV.predict(x_test),
    )
    return cfm_train, cfm_test

#11
def predict_tags(gridSearchCV, x_untagged):

    y_untagged_pred = gridSearchCV.predict(x_untagged)

    return y_untagged_pred


In [30]:
input="data.txt"
columns_data=["mensaje","etiqueta"]
data= read_data(input, columns_data)
x_tagged, y_tagged, x_untagged, y_untagged = group_data_for_label(data)
x_train, x_test, y_train, y_test = data_set_preparation(x_tagged, y_tagged)
analyzer=building_word_analyzer()
countVectorizer= instance_countvectorizer(analyzer)
pipeline = create_pipeline(countVectorizer)
gridSearchCV = create_instance_gridsearchCV(pipeline, x_train, y_train)
cfm_train, cfm_test = assess_model(gridSearchCV,x_train, x_test, y_train, y_test )
y_untagged_pred= predict_tags(gridSearchCV, x_untagged)


In [33]:
print(len(y_untagged_pred))
print(len(x_untagged))

13609
13609
